In [1]:
import pandas as pd
import numpy as np

# File Imports

In [11]:
full_movie_data = pd.read_csv("/Users/npop/Code/Projects/ML7641_Project/Official_Datasets/full_movie_data.csv")
reddit_comments = pd.read_csv("/Users/npop/Code/Projects/ML7641_Project/Official_Datasets/reddit_comments.csv")
youtube_comments = pd.read_csv("/Users/npop/Code/Projects/ML7641_Project/Official_Datasets/youtube_comments.csv")

/var/folders/jw/9mhjkj2530q0dcv2dbdg1zmm0000gn/T/ipykernel_47406/739861301.py:2: DtypeWarning: Columns (3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  reddit_comments = pd.read_csv("/Users/npop/Code/Projects/ML7641_Project/Official_Datasets/reddit_comments.csv")


# Selecting Valid Reddit Comments

In [12]:
reddit_comments.drop_duplicates(subset=['body', 'post_date'], inplace=True)
reddit_comments.dropna(subset=['post_date'], inplace=True)
reddit_comments.reset_index(drop=True, inplace=True)
full_movie_data.drop_duplicates(subset=['post_id'], inplace=True)
full_movie_data = full_movie_data[full_movie_data['Match'] == 1].copy(deep=True)
reddit_comments_v = reddit_comments[reddit_comments['post_id'].isin(full_movie_data['post_id'])].copy(deep=True)

In [133]:
reddit_comments_v['movie_title'] = "Not Set"
reddit_comments_v['movie_release_date'] = "Not Set"
reddit_comments_v['movie_actors'] = "Not Set"
reddit_comments_v.reset_index(drop=True, inplace=True)

In [134]:
def set_movie_info() -> None:
    for post_id in reddit_comments_v['post_id'].unique():
        movie_entry = full_movie_data[full_movie_data['post_id'].str.contains(post_id)]
        title = movie_entry['title'].values[0]
        release_date = movie_entry['release_date'].values[0]
        actors = movie_entry['Actors'].values[0]
        subset_filter = reddit_comments_v['post_id'].str.contains(post_id)
        reddit_comments_v.loc[subset_filter, "movie_title"] = title
        reddit_comments_v.loc[subset_filter, "movie_release_date"] = release_date
        reddit_comments_v.loc[subset_filter, "movie_actors"] = actors

In [135]:
#set_movie_info()

In [145]:
reddit_comments_v['post_date'] = pd.to_datetime(reddit_comments_v['post_date'])
reddit_comments_v['movie_release_date'] = pd.to_datetime(reddit_comments_v['movie_release_date'])

In [148]:
reddit_comments_v.dropna(subset=['movie_title'], inplace=True)
reddit_comments_v.reset_index(drop=True, inplace=True)

In [153]:
# Now we only want comments posted before the movie release date:
reddit_comments_v["valid_post_date"] = reddit_comments_v['post_date'] < reddit_comments_v['movie_release_date']

In [158]:
reddit_comments_v[reddit_comments_v['valid_post_date'] == True].to_csv("/Users/npop/Code/Projects/ML7641_Project/Official_Datasets/stage_2/reddit_comments_valid.csv", index=False)

In [159]:
full_movie_data.to_csv("/Users/npop/Code/Projects/ML7641_Project/Official_Datasets/stage_2/movie_data_valid.csv", index=False)

# Selecting Valid YouTube Comments

In [40]:
youtube_comments = pd.read_csv("/Users/npop/Code/Projects/ML7641_Project/Official_Datasets/youtube_comments.csv")
youtube_videos = pd.read_csv("/Users/npop/Code/Projects/ML7641_Project/Official_Datasets/youtube_videos.csv")

In [41]:
youtube_comments.drop_duplicates(subset=['comment_text', 'comment_date'], inplace=True)
youtube_comments.dropna(subset=['comment_date'], inplace=True)
youtube_comments.dropna(subset=['comment_text'], inplace=True)
youtube_comments.reset_index(drop=True, inplace=True)

In [42]:
def has_yt_link(text) -> bool:
    if type(text) != str:
        return False
    elif "youtu" in text:
        return True

def extract_youtube_id(url:str) -> str:
    """Extract the youtube video id from a youtube url
    
    Args:
        url (str): youtube url

    Returns:
        str: youtube video id
    """
    import re
    if not has_yt_link(url):
        print("Not a youtube link")
        return ""
    if "youtube" in url:
        match = re.search(r"(?<=v=)[a-zA-Z0-9_-]+", url)
        if match:
            return match.group(0)
    elif "youtu.be" in url:
        match = re.search(r"(?<=youtu.be/)[a-zA-Z0-9_-]+", url)
        if match:
            return match.group(0)
    else:
        print("Not a youtube link")
        return ""

In [ ]:
full_movie_data['yt_vid_id'] = full_movie_data['post_url_x'].apply(extract_youtube_id)

In [44]:
valid_yt_ids = full_movie_data['yt_vid_id'].unique()

In [46]:
youtube_comments_v = youtube_comments[youtube_comments['video_id'].isin(valid_yt_ids)].copy(deep=True)

In [49]:
youtube_comments_v.reset_index(drop=True, inplace=True)

In [60]:
youtube_comments_v['movie_title'] = "Not Set"
youtube_comments_v['movie_release_date'] = "Not Set"
youtube_comments_v['movie_actors'] = "Not Set"
youtube_comments_v.reset_index(drop=True, inplace=True)

In [61]:
def add_meta_to_yt_comments() -> None:
    for video_id in youtube_comments_v['video_id'].unique():
        movie_entry = full_movie_data[full_movie_data['yt_vid_id'].str.contains(video_id)]
        # Extract the movie information from full_movie_data
        title = movie_entry['title'].values[0]
        release_date = movie_entry['release_date'].values[0]
        actors = movie_entry['Actors'].values[0]
        # Update the comments dataframe only for the comments related to the current movie
        subset_filter = youtube_comments_v['video_id'].str.contains(video_id)
        youtube_comments_v.loc[subset_filter, "movie_title"] = title
        youtube_comments_v.loc[subset_filter, "movie_release_date"] = release_date
        youtube_comments_v.loc[subset_filter, "movie_actors"] = actors

In [64]:
full_movie_data['yt_vid_id'].fillna("No Link", inplace=True)

In [66]:
add_meta_to_yt_comments()

In [71]:
youtube_comments_v['comment_date'] = pd.to_datetime(youtube_comments_v['comment_date']).dt.date
youtube_comments_v['movie_release_date'] = pd.to_datetime(youtube_comments_v['movie_release_date']).dt.date

In [73]:
p = "/Users/npop/Code/Projects/ML7641_Project/Official_Datasets/stage_2/youtube_comments_valid.csv"
youtube_comments_v[youtube_comments_v['comment_date'] < youtube_comments_v['movie_release_date']].to_csv(p, index=False)